# (Q)分子構造+(R)理由+(A)物性データセットのLLMによる学習と予測
- Q&A: 融点データセットを使用
- R: 自分自身で考えさせて､正解のデータを学習させる

In [1]:
%reload_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from transformers import AutoTokenizer
import pandas as pd
import random
import copy
import glob
import json
from datetime import datetime
from llmchem.utils import mk_dir,clean_vram

#import clear_output

from IPython.display import clear_output

/home/setup/miniconda3/envs/chemllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#dataset settings
n_test=50 #number of testing data
n_train_check=50 #number of training data for checking (i.e., checking everything takes too long, so we check only a part of training data)
n_GPT_reasoning=30 # number of reasoning data made by GPT
n_generation_iterations=100   # trial numbers to generate new self reasoning data
max_generations=50

#model settings
model_name="mistralai/Mixtral-8x7B-Instruct-v0.1"
target_modules= [
    "lm_head",
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
    "gate",
    "w1",
    "w2",
    "w3"
]

model_name=f"meta-llama/Llama-2-7b-chat-hf"
target_modules= [
    #"embed_tokens",
    "lm_head",
    #"q_proj",
    #"k_proj",
    "v_proj",
    "o_proj",
    "gate_proj",
    "up_proj",
    #"down_proj",
]



#LoRA settings
r=32
lora_alpha=r
bit=16
#bit=8
#bit=4

#train settings
gradient_checkpointing =False
per_device_train_batch_size=1
epochs=3
lr=10**-5

#device settings
device_map="auto"

#dataset path
dataset_path="dataset/231225AutoReasoning/240117best_reason_record_3k.csv"

#project path
project_dir="results/projects/240117llama7b"

#reasoning options
error_threshold=30  # if abolute error is smaller than this, add to training data

In [3]:
mk_dir(project_dir)
mk_dir(project_dir+"/eval")
mk_dir(project_dir+"/self_reasoning")
mk_dir(project_dir+"/train")

In [4]:
from llmchem.model import init_model
from llmchem.train import train_model
from llmchem.eval import eval_model
from llmchem.reasoning import self_reasoning
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [5]:
#load base dataset

df=pd.read_csv(dataset_path)
dataset=df.to_dict(orient="records")
random.seed(0)
random.shuffle(dataset)

base_train_dataset=dataset[:-n_test]
example_reasoning_dataset=base_train_dataset[:n_GPT_reasoning]
test_dataset=dataset[-n_test:]

In [6]:
#Loop: training, evaluation, data generation
for generation in range(max_generations):
    clear_output()
    print(f"Generation: {generation}")
    #prepare train dataset

    ## reason data made by GPT4
    train_dataset=copy.deepcopy(example_reasoning_dataset)

    print(f"GPT-generated reasons: {len(train_dataset)}")

    ## reason data made by model itself
    for path in glob.glob(f"{project_dir}/self_reasoning/*.json"):
        with open(path) as f:
            train_dataset.append(json.load(f))

    print(f"All-generated reasons: {len(train_dataset)}")
    random.shuffle(train_dataset)

    #train model
    clean_vram()
    model=init_model(model_name, r, lora_alpha, target_modules, bit=bit,device_map=device_map)
    train_result=train_model(model,tokenizer,train_dataset,
                    project_dir=project_dir,
                    epochs=epochs,
                    lr=lr,
                    per_device_train_batch_size=per_device_train_batch_size,
                    gradient_checkpointing=gradient_checkpointing,
                    )

    #eval

    train_check_dataset=copy.deepcopy(train_dataset[:n_train_check])
    random.shuffle(train_check_dataset)
    train_eval_result=eval_model(model,tokenizer,train_check_dataset,
                                f"{project_dir}/eval",
                                n_prompt_examples=3,
                                prompt_dataset=example_reasoning_dataset,
                                prefix=f"train_{generation}"
                                )

    test_eval_result=eval_model(model,tokenizer,test_dataset,
                                f"{project_dir}/eval",
                                n_prompt_examples=3,
                                prompt_dataset=example_reasoning_dataset,
                                prefix=f"test_{generation}"
                                )

    #generate additional training data by self-reasoning
    self_reasoning(model,tokenizer,base_train_dataset,
                example_reasoning_dataset,project_dir,
                generation=generation,
                n_iterations=n_generation_iterations,
                error_threshold=error_threshold,
                n_max_trials=2)

Generation: 10
GPT-generated reasons: 30
All-generated reasons: 215
Using fp16 mode


Map: 100%|██████████| 215/215 [00:00<00:00, 10813.17 examples/s]


Step,Training Loss
100,1.527800
200,1.040300
300,0.856100
400,0.790100
500,0.715600
600,0.679200


  0%|          | 0/50 [00:00<?, ?it/s]

promlem 1 / 50


...

----


Considering the functional groups present in the molecule, we can predict the melting point of butanoic acid, 4-chloro-3-oxo-, ethyl ester as follows:

- Basic butanoic acid has a melting point of approximately -50°C.
- Chlorine atom adds +20°C due to increased polarity and potential for dipole-dipole interactions.
- Oxygen in the ester group can contribute +10°C due to increased molecular weight and potential for hydrogen bonding.
- The ethyl group adds +10°C due to increased molecular weight and potential for hydrogen bonding.

Considering the effects of each functional group, we estimate the predicted melting point for the ethyl ester of butanoic acid.
##Prediction: -20.0


#Problem
actual:  -8.0 predicted:  -20.0 err:  12.0
Considering the functional groups present in the molecule, we can predict the melting point of butanoic acid, 4-chloro-3-oxo-, ethyl ester as follows:

- Basic butanoic acid has a melting point of approximately -50°C.
- Chlorine atom adds +20°C due to inc

 64%|██████▍   | 64/100 [45:51<24:09, 40.26s/it]

----


The basic unit for comparison could be benzene, which has a melting point of 5.5 °C. The pentafluoromethoxy group (CF3O-) adds significant mass and electronegativity, which can increase the melting point due to stronger intermolecular forces. The fluorine atoms increase the melting point due to their high electronegativity and the potential for stronger London dispersion forces. The methoxy group (O-CH3) can form hydrogen bonds, which can increase the melting point. The presence of five fluorine atoms can significantly increase the melting point due to their high electronegativity and the potential for stronger London dispersion forces.
##Prediction: 120.0


#Problem
actual:  -37.0 predicted:  120.0 err:  157.0
The basic unit for comparison could be benzene, which has a melting point of 5.5 °C. The pentafluoromethoxy group (CF3O-) adds significant mass and electronegativity, which can increase the melting point due to stronger intermolecular forces. The fluorine atoms increase t

 65%|██████▌   | 65/100 [46:19<21:11, 36.34s/it]

----


The basic unit for 2-amino-6-fluoropyridine is pyridine, which has a melting point of approximately 40 degrees Celsius. The fluorine atom adds significant mass and electronegativity, which can increase the melting point due to stronger intermolecular forces. The amino group can form hydrogen bonds, which can also increase the melting point. However, the presence of the fluorine atom may disrupt the packing of the molecule, potentially reducing the melting point.
##Prediction: 95.0


#Problem
actual:  60.0 predicted:  95.0 err:  35.0
The basic unit for 2-amino-6-fluoropyridine is pyridine, which has a melting point of approximately 40 degrees Celsius. The fluorine atom adds significant mass and electronegativity, which can increase the melting point due to stronger intermolecular forces. The amino group can form hydrogen bonds, which can also increase the melting point. However, the presence of the fluorine atom may disrupt the packing of the molecule, potentially reducing the me